In [2]:
import pandas as pd
a = pd.DataFrame({'test': (1,2)})
a

,test
0,1
1,2


In [5]:
path = 'D:\\pyprojects\\rusjet\\Data_readyy.xlsx'
sheets_to_read = ['Flights', 'Planes', 'Airports']
sheets  = pd.read_excel(path, sheet_name=sheets_to_read, header=1)
flights, planes, airports = sheets['Flights'], sheets['Planes'], sheets['Airports']
#planes.head()
flights.head()
#airports.head()

,Дата,Борт,Маршрут,Flight Time,PAX АБ
0,2024-01-01,RA73528,UUWW-URSS,2.833333,5
1,2024-01-01,OELEV,USPP-URMN,3.133333,5
2,2024-01-01,RA73550,UUWW-GMMN,9.716667,5
3,2024-01-02,RA73550,GMMN-SBGL,9.416667,5
4,2024-01-02,RA67132,UUWW-ULLI,1.216667,0


In [12]:
#1ый и 2ой отчет проверка и приведение данных к нужному формату
months = 5
years = 2024
planes[['Скорость, тыс.км/час', 'Вес Взл, мтв', 'Пасс']] = planes[['Скорость, тыс.км/час', 'Вес Взл, мтв', 'Пасс']].dropna()
flights = flights.apply(lambda x: x if not (isinstance(x['Flight Time'], str) and ':' in x['Flight Time']) else None, axis=1)
#flights.rename({'PAX АБ': 'PAX AB'}, axis=1, inplace=True)
flights = flights.apply(lambda x: x if not (isinstance(x['PAX АБ'], str) and (x['PAX АБ'].isalnum)) else None, axis=1)
flights.dropna(subset=['PAX АБ', 'Flight Time'], inplace=True)
#flights['Flight Time'].unique()
#planes['Пасс'].unique()
#flights.head()
#planes.info()
flights.query('(Дата.dt.month <= @months) and (Дата.dt.year == @years)')

,Дата,Борт,Маршрут,Flight Time,PAX АБ
0,2024-01-01,RA73528,UUWW-URSS,2.833333,5.0
1,2024-01-01,OELEV,USPP-URMN,3.133333,5.0
2,2024-01-01,RA73550,UUWW-GMMN,9.716667,5.0
3,2024-01-02,RA73550,GMMN-SBGL,9.416667,5.0
4,2024-01-02,RA67132,UUWW-ULLI,1.216667,0.0
...,...,...,...,...,...
689,2024-05-30,RA67136,OMDW-UUWW,4.850000,3.0
690,2024-05-30,RA89194,UUDD-LLBG,5.550000,3.0
692,2024-05-30,RA73417,UMMS-URMG,3.116667,10.0
693,2024-05-30,RA73417,URMG-UUDD,2.183333,0.0


0     5
1     5
2     5
3     5
4     0
5     7
6     0
7     4
9     7
10    0
11    0
12    4
13    5
14    2
15    3
Name: PAX АБ, dtype: int32

In [4]:
# 1ый и 2ой отчет расчёт искомых значений
tmp = flights.merge(planes, right_on='Бортовой № ВС', left_on='Борт')
tmp1 = tmp['Скорость, тыс.км/час'] * tmp['Flight Time']
#tmp1 - Крейс. скор * Flight time
res1 = (tmp1 * tmp['Пасс']).sum() # Пассажироборот предельный																																																	
res2 = (tmp1 * tmp['PAX АБ']).sum() # Пассажироборот выполненный																																																	
res3 = tmp['PAX АБ'].sum() # Перевезено пассажиров
res4 = (tmp1 * tmp['Вес Взл, мтв']).sum() # Тоннокилометраж предельный																																																	
res5 = (res2 * 0.09).round(2) # Тоннокилометраж факт.
res5
 

753.9

In [5]:
from openpyxl import load_workbook

# Пример Series
res1ser = pd.Series([res1, res2, res3])
res11ser = pd.Series([res4, res5])

# Путь к существующему Excel файлу
path = 'C:\\rusjet\\Форма_отчета.xlsx'

# Прочитаем существующий Excel файл
book = load_workbook(path)
sheet_name = '10-ГА'
ws = book[sheet_name]

# Указываем начальную ячейку (например, строка 45 и колонка 109)
startrow = 45
startcol = 108

# Убедимся, что все предыдущие данные в ячейках в области записи удалены
for row in range(startrow, startrow + len(res1ser)):
    for col in range(startcol, startcol + 1):
        ws.cell(row=row, column=col).value = None

# Записываем данные Series в Excel
for i, value in enumerate(res1ser, start=startrow):
    cell = ws.cell(row=i, column=startcol)
    cell.value = value

# Добавление в таблицу второй части значений
startrow = 56
startcol = 108

for row in range(startrow, startrow + len(res11ser)):
    for col in range(startcol, startcol + 1):
        ws.cell(row=row, column=col).value = None

for i, value in enumerate(res11ser, start=startrow):
    cell = ws.cell(row=i, column=startcol)
    cell.value = value

# Сохраняем изменения в книге
book.save(path)